In [12]:
# notebooks/validation/02_validation_fusion_finale.ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("🔍 VALIDATION FINALE DE LA FUSION")
print("=" * 40)

# Chargement du dataset final
df = pd.read_csv("../../data/cleaned/sursaud_iqvia_fusionne_finale.csv")
df['date_debut_semaine'] = pd.to_datetime(df['date_debut_semaine'])

print(f"✅ Dataset final : {df.shape}")

# Validation complète
print("\n📊 VALIDATION COMPLÈTE")
print("=" * 30)

# 1. Vérification des valeurs manquantes
missing = df.isnull().sum().sum()
print(f"Valeurs manquantes : {missing}")

# 2. Vérification des classes d'âge
age_counts = df['classe_age'].value_counts()
print(f"\nClasses d'âge :")
for age, count in age_counts.items():
    print(f"   - {age}: {count:,} observations")

# 3. Vérification des ratios par classe d'âge
print(f"\nRatios doses/urgences par classe d'âge :")
for age in df['classe_age'].unique():
    age_data = df[df['classe_age'] == age]
    doses_mean = age_data['DOSES_J07E1'].mean()
    urgences_mean = age_data['taux_urgences_grippe'].mean()
    ratio = doses_mean / urgences_mean if urgences_mean > 0 else 0
    
    print(f"   - {age}: {ratio:.1f} doses/urgences")

# 4. Vérification de la cohérence temporelle
print(f"\nCohérence temporelle :")
for age in df['classe_age'].unique():
    age_data = df[df['classe_age'] == age]
    period_days = (age_data['date_debut_semaine'].max() - age_data['date_debut_semaine'].min()).days
    print(f"   - {age}: {period_days} jours")

# 5. Score de validation final
validation_score = 0
total_checks = 6

# Check 1: Volume
if len(df) > 1000:
    validation_score += 1
    print(f"\n✅ Volume suffisant : {len(df):,} observations")

# Check 2: Classes d'âge
if df['classe_age'].nunique() >= 5:
    validation_score += 1
    print(f"✅ Classes d'âge complètes : {df['classe_age'].nunique()}")

# Check 3: Valeurs manquantes
if missing == 0:
    validation_score += 1
    print(f"✅ Aucune valeur manquante")

# Check 4: Période
period_days = (df['date_debut_semaine'].max() - df['date_debut_semaine'].min()).days
if period_days > 1000:
    validation_score += 1
    print(f"✅ Période étendue : {period_days} jours")

# Check 5: Ratios cohérents
ratios = []
for age in df['classe_age'].unique():
    age_data = df[df['classe_age'] == age]
    ratio = age_data['DOSES_J07E1'].mean() / age_data['taux_urgences_grippe'].mean()
    ratios.append(ratio)

if all(0 < r < 10000 for r in ratios):
    validation_score += 1
    print(f"✅ Ratios cohérents")

# Check 6: Données réelles vs estimées
real_data = len(df[df['classe_age'].isin(['65 ans ou plus', 'moins de 65 ans'])])
estimated_data = len(df) - real_data
if real_data > 0 and estimated_data > 0:
    validation_score += 1
    print(f"✅ Mix données réelles ({real_data}) + estimées ({estimated_data})")

validation_percentage = (validation_score / total_checks) * 100
print(f"\n🎯 SCORE DE VALIDATION : {validation_score}/{total_checks} ({validation_percentage:.0f}%)")

if validation_percentage >= 80:
    print("🎉 FUSION VALIDÉE - Prêt pour Prophet !")
else:
    print("⚠️ FUSION PARTIELLEMENT VALIDÉE")

print(f"\n📊 RÉSUMÉ FINAL :")
print(f"   Observations : {len(df):,}")
print(f"   Classes d'âge : {df['classe_age'].nunique()}")
print(f"   Période : {period_days} jours")
print(f"   Données réelles : {real_data:,}")
print(f"   Données estimées : {estimated_data:,}")

🔍 VALIDATION FINALE DE LA FUSION
✅ Dataset final : (1284, 9)

📊 VALIDATION COMPLÈTE
Valeurs manquantes : 0

Classes d'âge :
   - 00-04 ans: 302 observations
   - 05-14 ans: 302 observations
   - 15-64 ans: 302 observations
   - Tous âges: 302 observations
   - 65 ans ou plus: 76 observations

Ratios doses/urgences par classe d'âge :
   - 65 ans ou plus: 738.6 doses/urgences
   - 00-04 ans: 500.0 doses/urgences
   - 05-14 ans: 500.0 doses/urgences
   - 15-64 ans: 1000.0 doses/urgences
   - Tous âges: 738.6 doses/urgences

Cohérence temporelle :
   - 65 ans ou plus: 1197 jours
   - 00-04 ans: 2107 jours
   - 05-14 ans: 2107 jours
   - 15-64 ans: 2107 jours
   - Tous âges: 2107 jours

✅ Volume suffisant : 1,284 observations
✅ Classes d'âge complètes : 5
✅ Aucune valeur manquante
✅ Période étendue : 2107 jours
✅ Ratios cohérents
✅ Mix données réelles (76) + estimées (1208)

🎯 SCORE DE VALIDATION : 6/6 (100%)
🎉 FUSION VALIDÉE - Prêt pour Prophet !

📊 RÉSUMÉ FINAL :
   Observations : 1,284
 